**Tarea: Creación de un modelo de clasificación para predecir problemas en procesos de un servidor**

In [85]:
## Importar librerias y funciones

import pandas as pd  # Libreria para manipulación y análisis de datos
import numpy as np   # Librería para cálculo numérico y operaciones con arreglos
import matplotlib.pyplot as plt  # Librería para crear gráficos y visualizaciones
import seaborn as sns  # Librería para visualizaciones estadísticas
from sklearn.model_selection import train_test_split  # Función para dividir datos en entrenamiento y prueba
from sklearn.preprocessing import StandardScaler  # Escalador para normalizar características

**1. Análisis Exploratorio de Datos (EDA)**:
Se usa para examinar un conjunto de datos, entender su estructura, identificar patrones y detectar valores faltantes antes de aplicar modelos de Machine Learning

In [86]:
## Cargar la data del archivo CSV
df = pd.read_csv('datos_procesos.csv',sep='|') # Lee el archivo CSV, especificando el separador
pd.set_option('display.max_columns', None) # Muestra todas las columnas que tiene la data
df.head() # muestra las primeras 5 filas de la data

,ID_Proceso,Uso_CPU,Uso_Memoria,Numero_Hilos,Tiempo_Ejecucion,Numero_Errores,Tipo_Proceso,Estado
0,1,37.454012,59.515562,16,8.184879,3,Aplicación,0
1,2,95.071431,36.471714,18,76.195256,8,Aplicación,0
2,3,73.199394,0.537562,21,46.991138,3,Aplicación,0
3,4,59.865848,56.108773,28,87.884325,4,Aplicación,0
4,5,15.601864,89.657041,39,4.622599,1,Sistema,0


In [87]:
# Solo muestra estadística descriptiva de las columnas numéricas
df.describe()

,ID_Proceso,Uso_CPU,Uso_Memoria,Numero_Hilos,Tiempo_Ejecucion,Numero_Errores,Estado
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,500000.500000,50.033449,49.947866,25.030640,50.043250,5.003483,0.064631
std,288675.278932,28.859113,28.851514,14.150159,28.829111,2.237005,0.245874
min,1.000000,0.000052,0.000038,1.000000,0.100083,0.000000,0.000000
25%,250000.750000,25.034021,24.999623,13.000000,25.074789,3.000000,0.000000
50%,500000.500000,50.055597,49.895350,25.000000,50.040186,5.000000,0.000000
75%,750000.250000,75.009673,74.924148,37.000000,75.030119,6.000000,0.000000
max,1000000.000000,99.999831,99.999938,49.000000,99.999992,19.000000,1.000000


In [88]:
# muestra la cantidad de valores nulos por columna si los hay
df.isnull().sum()

ID_Proceso          0
Uso_CPU             0
Uso_Memoria         0
Numero_Hilos        0
Tiempo_Ejecucion    0
Numero_Errores      0
Tipo_Proceso        0
Estado              0
dtype: int64

In [89]:
# cuenta cuantas veces aparece cada valor en la columna Estado
df['Estado'].value_counts()

Estado
0    935369
1     64631
Name: count, dtype: int64

In [90]:
# Cuenta el número total de duplicados
df.duplicated().sum()

np.int64(0)

**2. Preprocesamiento de datos**:
Es el proceso de preparar y limpiar los datos (eliminar duplicados, manejar valores nulos, convertir categorías) antes de aplicar un modelo de Machine Learning.

In [97]:
# Se aplica One-Hot Encoding para convertir la columna de texto en variable numérica
proceso= pd.get_dummies(df, columns=['Tipo_Proceso'])
proceso

,ID_Proceso,Uso_CPU,Uso_Memoria,Numero_Hilos,Tiempo_Ejecucion,Numero_Errores,Estado,Tipo_Proceso_Aplicación,Tipo_Proceso_Servicio,Tipo_Proceso_Sistema
0,1,37.454012,59.515562,16,8.184879,3,0,True,False,False
1,2,95.071431,36.471714,18,76.195256,8,0,True,False,False
2,3,73.199394,0.537562,21,46.991138,3,0,True,False,False
3,4,59.865848,56.108773,28,87.884325,4,0,True,False,False
4,5,15.601864,89.657041,39,4.622599,1,0,False,False,True
...,...,...,...,...,...,...,...,...,...,...
999995,999996,88.808083,46.057947,29,27.258890,4,0,False,True,False
999996,999997,43.731876,90.516047,36,5.368654,7,0,False,False,True
999997,999998,41.807198,68.311082,9,63.019433,4,0,False,False,True
999998,999999,42.867126,33.865907,48,4.543244,8,0,False,True,False


**3.Selección y División del conjunto de datos**

In [102]:
df_dummies = pd.get_dummies(df['Tipo_Proceso'], prefix='Tipo_Proceso')
df_dummies = df_dummies.astype(int) #Reemplaza los valores true/false por 1/0
df_final = pd.concat([df.drop(columns=['Tipo_Proceso']), df_dummies], axis=1) #concatena las columnas del df excluyendo la Tipo_Proceso
df_final

,ID_Proceso,Uso_CPU,Uso_Memoria,Numero_Hilos,Tiempo_Ejecucion,Numero_Errores,Estado,Tipo_Proceso_Aplicación,Tipo_Proceso_Servicio,Tipo_Proceso_Sistema
0,1,37.454012,59.515562,16,8.184879,3,0,1,0,0
1,2,95.071431,36.471714,18,76.195256,8,0,1,0,0
2,3,73.199394,0.537562,21,46.991138,3,0,1,0,0
3,4,59.865848,56.108773,28,87.884325,4,0,1,0,0
4,5,15.601864,89.657041,39,4.622599,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
999995,999996,88.808083,46.057947,29,27.258890,4,0,0,1,0
999996,999997,43.731876,90.516047,36,5.368654,7,0,0,0,1
999997,999998,41.807198,68.311082,9,63.019433,4,0,0,0,1
999998,999999,42.867126,33.865907,48,4.543244,8,0,0,1,0


**4.Construcción y evaluación de modelos**

In [58]:
#Regresion Logistica
#Se importan la librerias a utilizar
import pandas as pd  # Libreria para manipulación y análisis de datos
import numpy as np   # Librería para cálculo numérico y operaciones con arreglos
import matplotlib.pyplot as plt  # Librería para crear gráficos y visualizaciones
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [111]:
#cargar el conjunto de datos
dataset = df_final
dataset

,ID_Proceso,Uso_CPU,Uso_Memoria,Numero_Hilos,Tiempo_Ejecucion,Numero_Errores,Estado,Tipo_Proceso_Aplicación,Tipo_Proceso_Servicio,Tipo_Proceso_Sistema
0,1,37.454012,59.515562,16,8.184879,3,0,1,0,0
1,2,95.071431,36.471714,18,76.195256,8,0,1,0,0
2,3,73.199394,0.537562,21,46.991138,3,0,1,0,0
3,4,59.865848,56.108773,28,87.884325,4,0,1,0,0
4,5,15.601864,89.657041,39,4.622599,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
999995,999996,88.808083,46.057947,29,27.258890,4,0,0,1,0
999996,999997,43.731876,90.516047,36,5.368654,7,0,0,0,1
999997,999998,41.807198,68.311082,9,63.019433,4,0,0,0,1
999998,999999,42.867126,33.865907,48,4.543244,8,0,0,1,0


In [129]:
#Definir las variables
x = df_final['Numero_Errores']
y = df_final['Estado']

In [130]:
#Separo los datos de "train" en entrenamiento y prueba para probar los algoritmos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 )

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(800000, 8)
(200000, 8)
(800000,)
(200000,)


In [131]:
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)  ## Encontrar la media y varianza
X_test = escalar.transform(X_test)

In [132]:
# Desarrollo del Training
model = LogisticRegression()

## Supoort vector machines
## Decision Trees
## KNN
## Random

#Entrenamos el modelo

## antes del entrenamiento
model.fit(X_train, y_train)  ##Entranamiento

## despues del entrenamiento
# Encontro los w y el b

LogisticRegression()

In [133]:
#Realizamos una predicción
print(X_test.shape)
y_pred = model.predict(X_test)
print(y_pred.shape)
print(y_pred)

## predict_proba(X)

(200000, 8)
(200000,)
[0 0 0 ... 1 0 0]


In [134]:
#Verifico la matriz de Confusión
print(y_test.shape)
print(y_pred.shape)

matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

(200000,)
(200000,)
Matriz de Confusión:
[[187024      0]
 [     0  12976]]


In [135]:
precision = precision_score(y_test, y_pred)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
1.0


In [136]:
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
1.0


**Regresión con Árboles de Decisión**

In [147]:
#Importar librerias
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [158]:
#Definir las variables
x = df_final['Tiempo_Ejecucion']
y = df_final['Estado']

In [159]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [160]:
# Crear el modelo de árbol de decisión
model = DecisionTreeRegressor()

In [161]:
# Entrenar el modelo
model.fit(X_train, y_train)

DecisionTreeRegressor()

In [162]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [163]:
# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Error Cuadrático Medio:", mse)

Error Cuadrático Medio: 0.0


**LightGBM**

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = df_final.copy()
X = data[['ID_Proceso', 'Uso_CPU', 'Uso_Memoria', 'Numero_Hilos', 'Tiempo_Ejecucion', 'Numero_Errores',]]
y = data['Estado']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "max_depth": 6,
    "learning_rate": 0.3,
    "n_estimators": 1500
}

model = lgb.LGBMClassifier(**params)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


ModuleNotFoundError: No module named 'lightgbm'